In [1]:
import pandas as pd
import pickle
import math
from transformers import pipeline
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import requests
from time import sleep

In [2]:
data = pd.read_csv('../data/wiki_movie_plots_deduped.csv')

In [3]:
# data.head()

In [4]:
print(data.shape)

(34886, 8)


In [5]:
filtered_data = data[data['Genre'] != 'unknown']
filtered_data = filtered_data[filtered_data['Cast'].notna()]
filtered_data = filtered_data[filtered_data['Director'] != 'Unknown']

In [6]:
filtered_data.head(1)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
13,1907,Daniel Boone,American,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",biographical,https://en.wikipedia.org/wiki/Daniel_Boone_(19...,Boone's daughter befriends an Indian maiden as...


In [14]:
def wiki_scrape(url):
    # url = filtered_data['Wiki Page'].iloc[0]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # title = soup.find('h1', class_='firstHeading').text
    # Find and print the first paragraph
    # first_paragraph = soup.find('p').text
    # get short
    paragraphs = soup.find_all('p')
    for p in paragraphs:
        if len(p.text) >= 20:
            short = p.text
            break
    return short

In [15]:
filtered_data.index[878]

954

In [16]:
# url = filtered_data['Wiki Page'].iloc[954]
url = "https://en.wikipedia.org/wiki/Baby_(2015_Tamil_film)"
print(url)
print(wiki_scrape(url))

# paragraphs[1].text

https://en.wikipedia.org/wiki/Baby_(2015_Tamil_film)
Baby is a 2015 Indian Tamil-language horror film directed by D Suresh.[1] The story revolves around a separated couple who later reunite to protect their daughter from being haunted.[2]



In [ ]:
filtered_data['Plot'].iloc[954]

In [38]:
with open("../data/wiki_movies/checkpoint.pkl", 'rb') as f:
    data = pickle.load(f)

In [39]:
len(data)

TypeError: object of type 'NoneType' has no len()

In [ ]:
summary_list = []
scrape_plot = []
for idx, row in tqdm(filtered_data.iterrows(), total=len(filtered_data)):
    url = row['Wiki Page']
    short, plot = wiki_scrape(url)
    # print(short)
    # print(plot)
    summary_list.append(short)
    scrape_plot.append(plot)
    # sleep(1)
    break

In [ ]:
# import re
# import torch
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
# WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

# model_name = "csebuetnlp/mT5_multilingual_XLSum"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# def batch_summary_gen(model, tokenizer, article_texts, device):
#     input_ids = tokenizer(
#         [WHITESPACE_HANDLER(text) for text in article_texts],
#         return_tensors="pt",
#         padding="max_length",
#         truncation=True,
#         max_length=512
#     )["input_ids"]

#     output_ids = model.generate(
#         input_ids=input_ids.to(device),
#         max_length=64,
#         no_repeat_ngram_size=2,
#         num_beams=4
#     )

#     summary = []
#     for ids in output_ids:
#         summary.append(tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False))

#     # print(summary)
#     return summary

In [ ]:
# batch_size = 50
# summary_list = []
# for i in tqdm(range(0, filtered_data.shape[0], batch_size)):
#     plots = list(filtered_data['plot'].iloc[i:i+batch_size])
#     summaries = batch_summary_gen(model, tokenizer, plots, device)
#     print(summaries)
#     summary_list.extend(summaries)

In [ ]:
filtered_data["summary_text"] = summary_list
filtered_data["scraped_plot"] = scrape_plot
filtered_data.to_csv('../data/wiki_movies/filtered_wiki_movies_scraped.csv')